
# Продвинутые рекомендательные системы с Python

Добро пожаловать в тетрадку кода для создания продвинутых рекомендательных систем с Python. Это необязательная лекция, которую вы можете просмотреть. По причинам уровня применяемой математики и интенсивного использования SciPy, в настоящее время видео для этой лекции нет.

Рекомендательные системы обычно полагаются на большие наборы данных и, в особенности, должны быть организованы определенным образом. Поэтому, вместо проекта, соответствующего этой теме, мы проведем более интенсивный процесс пошагового руководства по созданию рекомендательных систем с Python с тем же набором данных Movie Lens Data Set.

*Примечание: Фактическая математика, лежащая в основе рекомендательных систем, имеет глубокие корни в линейной алгебре.*
___

## Используемые методы

Двумя наиболее распространёнными типами рекомендательных систем являются **Content-Based** (по содержанию или на основе контента) и **Collaborative Filtering (CF)** (коллаборативная фильтрация). 

* Коллаборативная фильтрация вырабатывает рекомендации, основанные на знании отношения пользователей к предметам, то есть использует "мудрость толпы" для рекомендации предметов. 
* Рекомендательные системы на основе контента фокусируются на атрибутах элементов и дают рекомендации на основе сходства между ними.

## Коллаборативная фильтрация (Collaborative Filtering)

В целом, коллаборативная фильтрация (CF) используется чаще, чем системы на основе контента, поскольку обычно она даёт лучшие результаты и относительно проста для понимания (с точки зрения общей реализации). Алгоритм имеет возможность самостоятельно изучать особенности, что означает, что он может начать самостоятельно изучать, какие особенности использовать. 

CF можно разделить на **Memory-Based Collaborative Filtering** (CF на основе памяти) и **Model-Based Collaborative filtering** (CF на основе модели). 

В этом руководстве мы будем претворять CF на основе модели, используя сингулярное разложение (SVD) и CF на основе памяти, вычислив косинусное сходство (cosine similarity). 

## Данные

Мы будем использовать известный набор данных MovieLens dataset, который является одним из наиболее распространенных наборов данных, используемых при реализации и тестировании рекомендательных систем. Он содержит 100 тысяч рейтингов фильмов от 943 пользователей, и предлагает выбор из 1682 фильмов.

Вы можете скачать набор данных [здесь](http://files.grouplens.org/datasets/movielens/ml-100k.zip), или просто используйте файл u.data, который уже включён в эту папку.

____
## Начало работы

Давайте импортируем некоторые из библиотек, которые нам понадобятся:

In [1]:
import numpy as np
import pandas as pd

Затем мы можем загрузить файл **u.data**, который содержит полный набор данных. Вы можете прочитать краткое описание набора данных [здесь](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).

Обратите внимание, как мы определяем аргумент separator для файла, разделенного табуляцией.

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

Давайте кратко рассмотрим данные.

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


Обратите внимание, что у нас есть только item_id, а не название фильма. Мы можем использовать csv-файл Movie_ID_Titles, чтобы извлечь названия фильмов и объединить их с этой dataframe:

In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


Затем объедините dataframes:

In [5]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


Теперь давайте взглянем на количество уникальных пользователей и фильмов.

In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 944
Num of Movies: 1682


## Разделение данных на обучающий и тестовый наборы

Системы рекомендаций по своей природе очень сложно оценивать, но в этом руководстве мы все-же покажем вам, как их оценивать. Для этого мы разделим наши данные на два набора. Тем не менее, мы не будем делать наши классические X_train,X_test,y_train,y_test split. Вместо этого мы можем просто сегментировать данные на два набора данных:

In [19]:
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

## Коллаборативная фильтрация на основе памяти (Memory-Based Collaborative Filtering)

Подходы к коллаборативной фильтрации на основе памяти можно разделить на два основных раздела: **user-item filtering** и **item-item filtering**. 

*user-item filtering* возьмет определенного пользователя, найдет пользователей, похожих на него, на основе сходства оценок, и порекомендует элемент, которые понравились пользователям схожим с ним/ней.

*item-item filtering*, в отличие, возьмет элемент, найдет пользователей, которым понравился этот элемент, включая другие элементы, которые также нравятся этим пользователям или аналогичным пользователям. Он берёт одни элементы и выводит другие элементы в качестве рекомендаций.

* *Item-Item Collaborative Filtering*: “Пользователям, которым понравился этот элемент, также понравилось …”
* *User-Item Collaborative Filtering*: “Пользователям,  похожим на вас, также понравилось…”

В обоих случаях вы создаёте матрицу пользовательских элементов, которая построена из всего набора данных.

Поскольку мы разделили данные на тестирование и обучение, нам нужно будет создать две матрицы ``[943 x 1682]``(все пользователи на все фильмы).

Матрица обучения содержит 75% рейтингов, а матрица тестирования - 25% рейтингов.

Пример матрицы user-item:
<img class="aligncenter size-thumbnail img-responsive" src="http://s33.postimg.org/ay0ty90fj/BLOG_CCA_8.png" alt="blog8"/>

После того, как вы построили матрицу user-item, рассчитайте сходство и создайте матрицу подобия (similarity matrix). 

Значения сходства между элементами в *Item-Item Collaborative Filtering* измеряются путем наблюдения всех пользователей, которые оценили оба элемента.  

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/i522ma83z/BLOG_CCA_10.png"/>

Для *User-Item Collaborative Filtering* значения сходства между пользователями измеряются путем наблюдения всех элементов, которые оценены обоими пользователями.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/mlh3z3z4f/BLOG_CCA_11.png"/>

Метрикой расстояния, обычно используемой в рекомендательных системах, является косинусное сходство, где рейтинги рассматриваются, как векторы в ``n``-мерном пространстве а сходство рассчитывается на основе угла между этими векторами. 
Косинусное сходство для пользователей *a* и *m* может быть рассчитан по следующей формуле: вы берете точечное произведение вектора пользователя *$u_k$* и вектора пользователя *$u_a$*, и делите его с помощью умножения Евклидовой длины векторов.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

Для вычисления сходства между элементами *m* и *b* используйте формулу:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Вашим первым шагом будет создание матрицы user-item. Поскольку у вас есть данные тестирования и обучения, вам нужно создать две матрицы.  

In [11]:
#Создайте две матрицы user-item: одну для обучения, а другую для тестирования
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

Вы можете использовать функцию [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) из sklearn, чтобы вычислить косинусное сходство. Обратите внимание, что диапазон выхода будет от 0 до 1, поскольку все оценки являются положительными.

In [12]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Следующий шаг -  прогнозирование. Вы уже создали матрицы коэффициента сходства:`user_similarity` и `item_similarity`, и поэтому сможете сделать прогноз, применив следующую формулу для CF на основе пользователя:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

Вы можете посмотреть на сходство между пользователями *k* и *a* , как на вес, который увеличивается в ответ на рейтинги аналогичного пользователя *a* (с поправкой на среднюю оценку этого пользователя). Вам нужно будет нормализовать его таким образом, чтобы рейтинги оставались между 1 и 5 и, в качестве последнего шага, суммировать средние рейтинги для пользователя, которого вы пытаетесь предсказать. 

Идея заключается в том, что некоторые пользователи могут иметь тенденцию всегда давать высокие или низкие оценки всем фильмам. Относительная разница в рейтингах, которые дают эти пользователи, важнее абсолютных значений. Для примера: предположим, пользователь *k* дает 4 звезды своим любимым фильмам, и 3 звезды - всем другим хорошим фильмам. Предположим теперь, что другой пользователь *t* оценивает фильмы, которые он/она любит, 5-ю звездами, а фильмы, на которых он/она заснул - 3-мя звездами. Эти два пользователя могут иметь очень схожий вкус, но по-разному относятся к системе рейтинга. 

При прогнозировании CF на основе элементов вам не нужно корректировать средний рейтинг пользователей, так как сам пользователь запроса используется для прогнозирования.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [13]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #Вы используете np.newaxis, так что mean_user_rating имеет тот же формат, что и рейтинги
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [14]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

### Оценка
Существует множество показателей оценки, но одной из самых популярных метрик, используемых для оценки точности прогнозируемых оценок, является *Среднеквадратическая ошибка (RMSE)*. 
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

Вы можете использовать функцию (MSE) [mean_square_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) из `sklearn`, где RMSE - это просто квадратный корень из MSE. Чтобы больше узнать о различных метриках оценки, вы можете взглянуть на [эту статью](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf). 

Поскольку вы хотите учесть только прогнозируемые рейтинги, находящиеся в наборе тестовых данных, отфильтруйте все остальные элементы в матрице прогнозирования с помощью `prediction[ground_truth.nonzero()]`. 

In [15]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [16]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 3.1262500764956056
Item-based CF RMSE: 3.452699732471426


Алгоритмы на основе памяти просты в реализации и обеспечивают приемлемое качество прогнозирования.  Недостатком CF на основе памяти является то, что он не масштабируется до реальных сценариев и не решает хорошо известную проблему холодного запуска, то есть когда в систему входит новый пользователь или новый элемент. Методы CF на основе моделей являются масштабируемыми и могут иметь более высокий уровень разреженности, чем модели на основе памяти, но также страдают, когда в систему попадают новые пользователи или элементы, не имеющие рейтингов. Я хотел бы поблагодарить Итана Розенталя за его [пост] http://blog.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/) о коллаборативной фильтрации на основе памяти (Memory-Based Collaborative Filtering). 

# Коллаборативная фильтрация на основе моделей

Коллаборативная фильтрация на основе моделей основана на **матричной факторизации (MF)**, которая получила более широкое распространение, главным образом в качестве метода обучения без учителя для декомпозиции скрытых переменных и снижения размерности. Матричная факторизация широко используется для рекомендательных систем, где она может лучше справляться с масштабируемостью и разреженностью, чем основанная на памяти CF. Матричная факторизация широко используется для рекомендательных систем, где она лучше справляется с масштабируемостью и разреженностью, чем основанная на памяти CF. Цель MF состоит в том, чтобы узнать скрытые предпочтения пользователей и скрытые атрибуты элементов из известных рейтингов (изучить особенности, которые описывают особенности рейтингов), чтобы затем предсказать неизвестные рейтинги через точечный продукт скрытых особенностей пользователей и элементов. 
Когда у вас очень разреженная матрица с большим количеством измерений, путем факторизации матрицы вы можете реструктурировать матрицу пользовательского элемента в структуру низкого ранга, и вы можете отобразить матрицу путем умножения двух матриц низкого ранга, где строки содержат скрытый вектор. Вы подгоняете эту матрицу, чтобы максимально приблизиться к исходной матрице, умножая матрицы низкого ранга друг на друга, что заполняет записи, отсутствующие в исходной матрице.

Рассчитаем уровень разреженности MovieLens dataset:

In [17]:
sparsity=round(1.0-len(df)/float(n_users*n_items),3)
print('The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%')

The sparsity level of MovieLens100K is 93.7%


Чтобы привести пример изученных скрытых предпочтений пользователей и элементов: предположим, что для набора данных MovieLens у вас есть следующая информация: _(user id, age, location, gender, movie id, director, actor, language, year, rating)_. Применяя матричную факторизацию, модель узнает, что важными особенностями пользователя являются _age group (under 10, 10-18, 18-30, 30-90)_, _location_ и _gender_, что касается особенностей фильма, она узнает, что _decade_, _director_ and _actor_  являются наиболее важными. Теперь, если вы посмотрите на информацию, которую вы сохранили, такой особенности, как _decade_, не существует, но модель может учиться самостоятельно. Важным аспектом является то, что модель CF использует только данные (user_id, movie_id, rating) для изучения скрытых особенностей. Если данных недостаточно, модель CF на основе модели будет предсказывать неудачно, так как ей будет сложнее изучить скрытые особенности. 

Модели, использующие как рейтинги, так и особенности контента, называются **Гибридные Рекомендательные Системы** (**Hybrid Recommender Systems**), в которых совмещаются как коллаборативная фильтрация, так и модели на основе контента. Гибридные рекомендательные системы обычно показывают более высокую точность, чем коллаборативная фильтрация или модели на основе контента сами по себе: они способны лучше решать проблему холодного запуска, поскольку, если у вас нет оценок для пользователя или элемента, вы можете использовать метаданные от пользователя или элемента для прогнозирования.

### SVD (Сингулярное разложение)
Хорошо известным методом матричной факторизации является **Сингулярное разложение (SVD)**. Коллаборативная фильтрация может быть сформулирована путем аппроксимации матрицы `X` с использованием разложения по сингулярным значениям. Команда-победитель конкурса Netflix Prize использовала модели факторизации матрицы SVD для выработки рекомендаций по продукту, для получения дополнительной информации я рекомендую прочитать статьи: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) and [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).
Общее уравнение может быть выражено следующим образом:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Дана `m x n` матрица `X`:
* *`U`* это *`(m x r)`* ортогональная матрица
* *`S`* это *`(r x r)`* диагональная матрица с неотрицательными действительными числами на диагонали
* *V^T* это *`(r x n)`* ортогональная матрица

Элементы на диагональной в `S` известны, как *сингулярные значения `X`*. 


Матрица *`X`* может быть разложена на *`U`*, *`S`* и *`V`*. Матрица *`U`* представляет векторы особенностей, соответствующие пользователям в скрытом пространстве особенностей, а матрица *`V`* представляет векторы особенностей, соответствующие элементам в пространстве скрытых особенностей.
<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/kwgsb5g1b/BLOG_CCA_5.png"/>

Теперь вы можете сделать прогноз, взяв точечное произведение *`U`*, *`S`* и *`V^T`*.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="http://s33.postimg.org/ch9lcm6pb/BLOG_CCA_4.png"/>

In [18]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#Получите компоненты SVD из train matrix. Выберите k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 2.7155526399659697


В случае небрежного довольствования лишь относительно небольшим количеством известных записей приводит к переподгонке. SVD может быть очень медленным и ресурсоемким. Более поздние работы минимизируют квадратическую ошибку, применяя чередующийся метод наименьших квадратов или стохастический градиентный спуск, и используют термины регуляризации для предотвращения переобучения. Чередующиеся методы наименьших квадратов и стохастического градиентного спуска для CF будут рассмотрены в следующих уроках.

Обзор:

* Мы рассмотрели, как реализовать простые методы **Коллаборативной фильтрации**, как на основе памяти CF, так и на основе модели CF.
* **Модели на основе памяти** основаны на сходстве предметов или пользователей, где мы используем косинусное сходство.
* **Основанный на модели CF** основана на факторизации матрицы, где мы используем SVD для факторизации матрицы.
* Построение рекомендательных систем, которые хорошо работают в сценариях холодного запуска (где мало данных о новых пользователях и элементах) остается проблемой. Стандартный метод коллаборативной фильтрации плохо работает в таких условиях.

## Желаете еще?

Если вы хотите провести собственный анализ системы рекомендаций, проверьте эти наборы данных. Примечание: В большинстве случаев файлы достаточно велики, не все ссылки могут оставаться активными, чтобы хранить данные, но большинство из них все еще работают. Или просто Google, чтоб найти свой собственный набор данных!

**Рекомендации фильмов:**

MovieLens - Наборы данных рекомендаций по фильмам http://www.grouplens.org/node/73

Yahoo! - Наборы данных рейтингов фильмов, музыки и изображений http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Jester - Наборы данных рейтингов фильмов (набор данных для коллаборативной фильтрации) http://www.ieor.berkeley.edu/~goldberg/jester-data/

Корнеллский университет - Данные обзоров фильмов для использования в экспериментах анализа тональности текста (sentiment-analysis experiments) http://www.cs.cornell.edu/people/pabo/movie-review-data/

**Музыкальные рекомендации:**

Last.fm - Наборы музыкальных рекомендаций http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/index.html

Yahoo! - Наборы данных рейтингов фильмов, музыки и изображений http://webscope.sandbox.yahoo.com/catalog.php?datatype=r

Audioscrobbler - Наборы музыкальных рекомендаций http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

Amazon - Рекомендации аудио CD http://131.193.40.52/data/

**Рекомендация по книгам:**

Institut für Informatik, Universität Freiburg - Наборы данных рейтингов книг http://www.informatik.uni-freiburg.de/~cziegler/BX/

Рекомендация по еде:

Chicago Entree - Наборы данных пищевых рейтингов http://archive.ics.uci.edu/ml/datasets/Entree+Chicago+Recommendation+Data

Рекомендация По Товарам:

**Рекомендация здравоохранения:**

Дом престарелых - Набор данных рейтингов провайдеров http://data.medicare.gov/dataset/Nursing-Home-Compare-Provider-Ratings/mufm-vy8d

Рейтинги больниц - опрос пациентов/больничный опыт http://data.medicare.gov/dataset/Survey-of-Patients-Hospital-Experiences-HCAHPS-/rj76-22dk

**Рекомендация по знакомству:**

www.libimseti.cz - Рекомендация сайтов 
знакомств (коллаборативная фильтрация) http://www.occamslab.com/petricek/data/

Рекомендация научной статьи:

Национальный университет Сингапура - Рекомендация научных статей http://www.comp.nus.edu.sg/~sugiyama/SchPaperRecData.html

# Прекрасная работа!